In [91]:
import cv2
import numpy as np
from retinaface import RetinaFace
from deepface import DeepFace
from skimage import exposure


# Function for super-resolution (using bicubic interpolation as an example)
def super_resolution(image, scale_factor=2):
    return cv2.resize(image, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_CUBIC)


# Function for contrast enhancement using CLAHE
def enhance_contrast(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    limg = cv2.merge((cl, a, b))
    return cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)


# Load the image
image_path = "./datasets/people/BD09DAF0-84DC-437F-8230-3323B7614C0F_1_105_c.jpeg"
image = cv2.imread(image_path)

# Detect faces using RetinaFace
faces = RetinaFace.detect_faces(image_path)

def get_embeddings(image_path):
    embeddings = []
    image = cv2.imread(image_path)
    faces = RetinaFace.detect_faces(image)
    for face_key in faces:
        face = faces[face_key]
        facial_area = face["facial_area"]

        # Extract face ROI
        x, y, w, h = facial_area
        face_image = image[y : y + h, x : x + w]

        # Apply super-resolution
        face_image = super_resolution(face_image)

        # Enhance contrast
        face_image = enhance_contrast(face_image)

        # Get face embedding using DeepFace (with ArcFace model)
        embedding = DeepFace.represent(face_image, model_name="ArcFace", enforce_detection=False)

        embeddings.append(embedding)

    return embeddings

faces1 = get_embeddings(image_path)
print(f"Number of faces detected: {len(faces1)}")
print(f"Shape of each embedding: {np.array(faces1[0]).shape}")

Number of faces detected: 2
Shape of each embedding: (1,)


In [92]:
image1 = Image.open(image_path)
image2 = Image.open("./datasets/people/61705984-3729-47CB-A20B-DD1CFF9B9BA7_1_105_c.jpeg")

In [125]:
faces1 = get_embeddings("./datasets/people/028C2197-6F21-4DBB-A868-AC93EE45A1F9_4_5005_c.jpeg")
faces2 = get_embeddings("./datasets/people/61705984-3729-47CB-A20B-DD1CFF9B9BA7_1_105_c.jpeg")

In [99]:
def crop_resize(img, box, image_size):
    x = box[0]
    y = box[1]
    w = box[2] - x
    h = box[3] - y

    out = img.crop(box).copy().resize((image_size, image_size), Image.BILINEAR)
    return out


def get_size(img):
    if isinstance(img, np.ndarray):
        return img.shape[1::-1]
    else:
        return img.size


def extract_face(img, box, image_size=160, margin=0):
    margin = [
        margin * (box[2] - box[0]) / (image_size - margin),
        margin * (box[3] - box[1]) / (image_size - margin),
    ]
    raw_image_size = get_size(img)
    box = [
        int(max(box[0] - margin[0] / 2, 0)),
        int(max(box[1] - margin[1] / 2, 0)),
        int(min(box[2] + margin[0] / 2, raw_image_size[0])),
        int(min(box[3] + margin[1] / 2, raw_image_size[1])),
    ]

    face = crop_resize(img, box, image_size)

    face = np.array(face, dtype=np.float32)

    return face

In [126]:
emb1 = np.array(faces1[0][0]['embedding'])
emb2 = np.array(faces2[0][0]['embedding'])

In [127]:
from sentence_transformers import util

In [128]:
util.cos_sim(emb1, emb2)

tensor([[0.6702]], dtype=torch.float64)